# Task I - Using linear regression for forecasting

### 1.Data preparation: Use one of the methods described in Project 1 (Advanced), Task 1 to pre-process the trace. Remove possible outliers. Reduce the dimensionality of the feature space to k = 16 using tree-based feature selection. Then, split the processed trace into training and test samples (x(t),y(t)) by assigning the samples with t < T to the training set and t ≥T to the test set. T is chosen so that the training set contains 70% of the samples.

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

%matplotlib inline

X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')
X.index = pd.to_datetime(X['TimeStamp'])
Y.index = pd.to_datetime(Y['TimeStamp'])
X_dropped = X.drop(labels=["Unnamed: 0", "TimeStamp"], axis=1, inplace=False)
Y_dropped = Y.drop(labels=["Unnamed: 0", "TimeStamp"], axis=1, inplace=False)
X_preprocessed = pd.DataFrame()
X_tmp = preprocessing.StandardScaler().fit_transform(X_dropped)
for i, n in enumerate(X_dropped):
        X_preprocessed[n] = X_tmp[:, i]
#print(X_preprocessed)



/home/anisha/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/anisha/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [54]:
#converting Y_dropped to make it of the same form as X_preprocessed
Y_new = pd.DataFrame()
Y_tmp=Y_dropped.to_numpy()
for i, n in enumerate(Y_dropped):
        Y_new[n] = Y_tmp[:, i]  
# outlier rejection   
remove = []
for i in X_preprocessed:
    for j in range(len(X_preprocessed[i])):
        if j not in remove and abs(X_preprocessed[i][j]) > 20:
            remove.append(j)
X_clean = X_preprocessed.drop(labels=remove, axis=0, inplace=False)
Y_clean = Y_new.drop(labels=remove, axis=0, inplace=False)

print("Number of dropped samples: ", (len(remove)))

Number of dropped samples:  1216


In [84]:
#Reduce the dimensionality of the feature space 
tree = ExtraTreesRegressor(n_estimators=100, n_jobs=20)
tree = tree.fit(X_clean, Y_clean)
model = SelectFromModel(tree,prefit=True,max_features=16,threshold=-np.inf)
X_fs = model.transform(X_clean)
top_features = []
get_feat = model.get_support()
for i, n in enumerate(X_clean):
    if get_feat[i]:
        top_features.append(n)
feature_name = []
for i in range(16):
    feature_name.append('feature'+str(i+1)+" : "+top_features[i])
X_latest = pd.DataFrame(data = X_fs,columns= feature_name)
Y_latest = pd.DataFrame(data = Y_clean,columns= Y_dropped.columns)
X_latest

,feature1 : 1_i127_intr.s,feature2 : 4_TxPacktes,feature3 : 4_TxBytes,feature4 : 15_RxPacktes,feature5 : 15_TxPacktes,feature6 : 17_RxPacktes,feature7 : 17_TxPacktes,feature8 : 30_RxPacktes,feature9 : 40_TxPacktes,feature10 : 41_TxPacktes,feature11 : 4_RxPacktes.1,feature12 : 4_TxBytes.1,feature13 : 15_RxPacktes.1,feature14 : 17_TxPacktes.1,feature15 : 40_TxPacktes.1,feature16 : 40_RxBytes.1
0,0.855128,1.238908,1.194544,1.247687,1.290084,1.299794,1.238812,1.201219,1.263907,1.224345,1.299455,1.194544,1.247687,1.238812,1.263907,1.110672
1,0.879198,1.115817,1.240098,1.079864,0.999243,0.974463,1.077493,1.111918,1.012137,1.111521,1.025629,1.240098,1.079864,1.077493,1.012137,1.234329
2,0.566292,1.127852,1.187528,1.132818,1.110978,1.117959,1.135340,1.174922,1.126207,1.141096,1.138740,1.187528,1.132818,1.135340,1.126207,1.215636
3,0.746815,1.166147,1.318807,1.068458,1.108240,1.110839,1.048163,1.154651,1.061839,1.114260,1.063468,1.318807,1.068458,1.048163,1.061839,1.254974
4,1.143965,1.173259,1.119624,1.169071,1.209569,1.208877,1.205407,1.171635,1.229686,1.206819,1.235169,1.119624,1.169071,1.205407,1.229686,1.179406
5,1.083791,1.080804,1.169513,1.086381,1.114812,1.117411,1.055495,1.133833,1.095653,1.119189,1.087881,1.169513,1.086381,1.055495,1.095653,1.211234
6,1.011581,1.138794,1.143921,1.136891,1.167394,1.162322,1.174447,1.138763,1.128652,1.057300,1.210757,1.143921,1.136891,1.174447,1.128652,1.157781
7,0.939372,1.183106,1.198475,1.134447,1.200257,1.193541,1.118638,1.139311,1.150651,1.128499,1.134671,1.198475,1.134447,1.118638,1.150651,1.048923
8,0.951407,1.219213,1.156582,1.210619,1.211760,1.210520,1.220073,1.265866,1.237833,1.208462,1.172510,1.156582,1.210619,1.220073,1.237833,1.122646
9,0.710710,1.145358,1.186189,1.159702,1.083044,1.191898,1.188705,1.180948,1.176724,1.166838,1.163559,1.186189,1.159702,1.188705,1.176724,1.197614


In [93]:
#splitting the processed trace into training and test samples and time index sorting
X_train, X_test, Y_train, Y_test = train_test_split(X_latest, Y_latest, test_size=0.3, shuffle = False)
print(X_train.shape,"(70% of the samples in training set and 16 features)")
X_train = X_train.sort_index(axis = 0)
X_test = X_test.sort_index(axis = 0)
Y_train = Y_train.sort_index(axis = 0)
Y_test = Y_test.sort_index(axis = 0)

(9285, 16) (70% of the samples in training set and 16 features)
